In [63]:
import pandas as pd
df = pd.read_csv("relativeLengths.csv").set_index("Names")
print(df)
print(df["Index"].sum())
print(df["Index"].iloc[3])

             Index  Middle   Third  Fourth
Names                                     
METATOBASE  132.59  132.59  132.59  132.59
PIPTOMETA    30.19   33.22   28.72   22.89
DIPPIP       29.22   32.97   29.25   21.30
ENDDIP       31.95   33.81   33.84   30.70
223.95
31.95


In [1]:
import pyvista as pv
import numpy as np

topRadii = 4
bottomRadii = 3
fingerBodyHeight = 4
fingerBodyWidth = 6
fingerBodyLength = 10
resolution = 9
holeRadius = 0.5

def generateFingerPart(resolution, topRadii, bottomRadii, fingerBodyHeight, fingerBodyWidth, holeRadius, fingerBodyLength):
    def endFace(resolution, topRadii, bottomRadii, fingerBodyHeight, fingerBodyWidth, holeRadius):
        newPoints = []
        faces = []
        thetaInterval = np.pi/(resolution)
        for i in range(2):
            batch = []
            theta = 0
            while theta < np.pi/2:
                radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2-bottomRadii:
                    batch.append((radius*np.cos(theta)*((-1)**i), -radius*np.sin(theta), 0))
                    theta += thetaInterval
                    radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2:
                    batch.append((radius*np.cos(theta)*((-1)**i), -radius*np.sin(theta), 0))
                    theta += thetaInterval
                radius = fingerBodyHeight/2/np.sin(theta)  
                while radius*np.cos(theta)>0:
                    batch.append((radius*np.cos(theta)*((-1)**i), -radius*np.sin(theta), 0))
                    theta += thetaInterval  
                    radius = fingerBodyHeight/2/np.sin(theta)
            if i%2==1:
                batch.reverse()
                batch.pop(0)
            newPoints.extend(batch)  
        newPoints.pop(0)
        newPoints.pop()
        newPoints.append((0,0,0))
        for p in range(len(newPoints)-1):
            faces.append((3, p, len(newPoints)-1, p+1))
        for i in range(2):
            batch = []
            theta = 0
            while theta < np.pi/2:
                radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2-topRadii:
                    batch.append((radius*np.cos(theta)*((-1)**i), radius*np.sin(theta), radius*np.sin(theta)))
                    theta += thetaInterval
                    radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2:
                    batch.append((radius*np.cos(theta)*((-1)**i), radius*np.sin(theta), radius*np.sin(theta)))
                    theta += thetaInterval
                radius = fingerBodyHeight/2/np.sin(theta)  
                while radius*np.cos(theta)>0:
                    batch.append((radius*np.cos(theta)*((-1)**i), radius*np.sin(theta), radius*np.sin(theta)))
                    theta += thetaInterval  
                    radius = fingerBodyHeight/2/np.sin(theta)  
            if i%2==1:
                batch.reverse()
                batch.pop(0)
            newPoints.extend(batch)
        for i in range(resolution+1):
            theta = 2*np.pi/resolution*i+(3*np.pi/2)
            newPoints.append((holeRadius*np.cos(theta), holeRadius*np.sin(theta)+fingerBodyHeight/4, holeRadius*np.sin(theta)+fingerBodyHeight/4)) 
        offset = resolution-2
        faces.append((3, offset-1, offset, resolution+offset))
        faces.append((3, 0, offset, offset+1))
        for i in range(resolution+1):
            faces.append((3, i+offset, (i+1)%(resolution+1)+offset, (i)%(resolution+1)+resolution+offset+1))
            faces.append((3, (i)%(resolution+1)+resolution+offset+1, (i+1)%(resolution+1)+resolution+offset+1, (i+1)%(resolution+1)+offset))
        return (np.array(newPoints), faces)

    faces = []
    totalVertices = []
    frontVertices, facesFront = endFace(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius)
    backVertices, facesBack = endFace(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius)
    adjustedFrontFaceVertices = []
    for vertice in frontVertices:
        adjustedFrontFaceVertices.append((vertice[0], vertice[1], vertice[2]-fingerBodyLength/2))
    totalVertices.extend(adjustedFrontFaceVertices)
    faces.extend(facesFront)
    adjustedBackFaceVertices = []
    for vertice in backVertices:
        adjustedBackFaceVertices.append((vertice[0], vertice[1], -1*vertice[2]+fingerBodyLength/2))
    adjustedBackFaces = []
    for idx in range(len(facesBack)):
        newFace = (3, facesBack[idx][1]+len(frontVertices), facesBack[idx][2]+len(frontVertices), facesBack[idx][3]+len(frontVertices))
        adjustedBackFaces.append(newFace)
    totalVertices.extend(adjustedBackFaceVertices)
    faces.extend(adjustedBackFaces)
    for idx in range(resolution-3):
        faces.append([3, idx, idx+1, idx+len(frontVertices)])
        faces.append([3, idx+len(frontVertices), idx+len(frontVertices)+1, idx+1])
    offset = resolution-1
    for idx in range(resolution-1):
        faces.append([3, idx+offset, idx+offset+1, idx+offset+len(frontVertices)])
        faces.append([3, idx+offset+len(frontVertices), idx+offset+len(frontVertices)+1, idx+offset+1])

    faces.append([3, 0, len(frontVertices), len(frontVertices)+resolution-1])
    faces.append([3, 0, len(frontVertices)+resolution-1, resolution-1])
    faces.append([3, resolution-3, len(frontVertices)+resolution-3, len(frontVertices)+resolution+resolution-2])
    faces.append([3, resolution-3, len(frontVertices)+resolution+resolution-2, resolution+resolution-2])

    offset = resolution+resolution-1
    for i in range(resolution+1):
        faces.append([3, offset+i, offset+((i+1)%(resolution+1)), len(frontVertices)+offset+(i%(resolution+1))])
        faces.append([3, offset+((i+1)%(resolution+1)), len(frontVertices)+offset+(i%(resolution+1)), len(frontVertices)+offset+((i+1)%(resolution+1))])
    mesh = pv.PolyData(totalVertices, faces).triangulate()
    return mesh

def move_and_align_mesh(mesh, new_center, new_align_vector, new_normal):
    otherNormal = np.cross(new_align_vector, new_normal)
    transformation_matrix = np.eye(4)
    A = [new_align_vector/np.linalg.norm(new_align_vector), new_normal/np.linalg.norm(new_normal), otherNormal/np.linalg.norm(otherNormal)]
    transformation_matrix[0:3, 0:3] =np.linalg.inv(A)
    mesh = mesh.transform(transformation_matrix)
    translation_vector = new_center - np.array([0,0,0], dtype=float)
    mesh = mesh.translate(translation_vector)
    return mesh

thingy = generateFingerPart(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius, fingerBodyLength=fingerBodyLength)
# thingy = move_and_align_mesh(thingy, np.array([0,0,0], dtype=float), np.array([1,1,0], dtype=float), np.array([1,0,3], dtype=float))
oldThingy = generateFingerPart(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius, fingerBodyLength=fingerBodyLength)

pl = pv.Plotter()
pl.add_mesh(thingy)
pl.add_mesh(oldThingy)
pl.show()

Widget(value="<iframe src='http://localhost:53052/index.html?ui=P_0x217b11bb3d0_0&reconnect=auto' style='width…

In [2]:
import pyvista as pv
import numpy as np

topRadii = 1
bottomRadii = 1
fingerBodyHeight = 6
fingerBodyWidth = 7
proximalLength = 9
distalLength = 12
resolution = 9
holeRadius = 0.5

def generateFingerPart(resolution, topRadii, bottomRadii, fingerBodyHeight, fingerBodyWidth, holeRadius, PIP, DIP, end):
    def endFace(resolution, topRadii, bottomRadii, fingerBodyHeight, fingerBodyWidth, holeRadius, angleBottom, angleTop, center):
        if resolution%2!=1:
            print("Resolution must be an odd number")
            return None
        newPoints = []
        faces = []
        thetaInterval = np.pi/(resolution)
        for i in range(2):
            batch = []
            theta = 0
            while theta < np.pi/2:
                radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2-bottomRadii:
                    if np.tan(angleBottom) == 0:
                        batch.append((radius*np.cos(theta)*((-1)**i)+center[0], -radius*np.sin(theta)+center[1], center[2]))
                    else:
                        batch.append((radius*np.cos(theta)*((-1)**i)+center[0], -radius*np.sin(theta)+center[1],-radius*np.sin(theta)/np.tan(angleBottom)+center[2]))
                    theta += thetaInterval
                    radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2:
                    if np.tan(angleBottom) == 0:
                        batch.append((radius*np.cos(theta)*((-1)**i)+center[0], -radius*np.sin(theta) + center[1], center[2]))
                    else:
                        batch.append((radius*np.cos(theta)*((-1)**i)+center[0], -radius*np.sin(theta) + center[1], -radius*np.sin(theta)/np.tan(angleBottom)+center[2]))
                    theta += thetaInterval
                radius = fingerBodyHeight/2/np.sin(theta)  
                while radius*np.cos(theta)>0:
                    if np.tan(angleBottom)==0:
                        batch.append((radius*np.cos(theta)*((-1)**i)+center[0], -radius*np.sin(theta) + center[1], center[2]))
                    else:
                        batch.append((radius*np.cos(theta)*((-1)**i)+center[0], -radius*np.sin(theta) + center[1], -radius*np.sin(theta)/np.tan(angleBottom)+center[2]))
                    theta += thetaInterval  
                    radius = fingerBodyHeight/2/np.sin(theta)
            if i%2==1:
                batch.reverse()
                batch.pop(0)
            newPoints.extend(batch)  
        newPoints.pop(0)
        newPoints.pop()
        newPoints.append(center)
        for p in range(len(newPoints)-1):
            faces.append((3, p, len(newPoints)-1, p+1))
        for i in range(2):
            batch = []
            theta = 0
            while theta < np.pi/2:
                radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2-topRadii:
                    batch.append((radius*np.cos(theta)*((-1)**i)+center[0], radius*np.sin(theta)+center[1], radius*np.sin(theta)/np.tan(angleTop)+center[2]))
                    theta += thetaInterval
                    radius = fingerBodyWidth/2/np.cos(theta) 
                while radius*np.sin(theta)<fingerBodyHeight/2:
                    batch.append((radius*np.cos(theta)*((-1)**i)+center[0], radius*np.sin(theta)+center[1], radius*np.sin(theta)/np.tan(angleTop)+center[2]))
                    theta += thetaInterval
                radius = fingerBodyHeight/2/np.sin(theta)  
                while radius*np.cos(theta)>0:
                    batch.append((radius*np.cos(theta)*((-1)**i)+center[0], radius*np.sin(theta)+center[1], radius*np.sin(theta)/np.tan(angleTop)+center[2]))
                    theta += thetaInterval  
                    radius = fingerBodyHeight/2/np.sin(theta)  
            if i%2==1:
                batch.reverse()
                batch.pop(0)
            newPoints.extend(batch)
        for i in range(resolution+1):
            theta = 2*np.pi/resolution*i+(3*np.pi/2)
            newPoints.append((holeRadius*np.cos(theta) + center[0], holeRadius*np.sin(theta)+fingerBodyHeight/4 + center[1],(holeRadius*np.sin(theta)+fingerBodyHeight/4)/np.tan(angleTop) + center[2])) 
        offset = resolution-2
        faces.append((3, offset-1, offset, resolution+offset))
        faces.append((3, 0, offset, offset+1))
        for i in range(resolution+1):
            faces.append((3, i+offset, (i+1)%(resolution+1)+offset, (i)%(resolution+1)+resolution+offset+1))
            faces.append((3, (i)%(resolution+1)+resolution+offset+1, (i+1)%(resolution+1)+resolution+offset+1, (i+1)%(resolution+1)+offset))
        return (np.array(newPoints), faces)

    faces = []
    totalVertices = []
    numFaces = 5
    proximalStartVertices, proximalStartFaces = endFace(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius, angleBottom=0, angleTop=45, center=PIP)
    proximalEndVertices, na = endFace(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius, angleBottom=-45, angleTop=-45, center=DIP)
    distalEndVertices, distalEndFaces = endFace(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=4, fingerBodyWidth=4, holeRadius=holeRadius, angleBottom=-45, angleTop=-45, center=end)
    totalVertices.extend(proximalStartVertices)
    totalVertices.extend(proximalEndVertices)
    faces.extend(proximalStartFaces) 
    for i in range(numFaces-2):
        print(fingerBodyHeight-(fingerBodyHeight-2)/(numFaces-2)*i)
        transitionalVertices, na = endFace(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight-(fingerBodyHeight-4)/(numFaces-1)*(i+1), fingerBodyWidth=fingerBodyWidth-(fingerBodyWidth-4)/(numFaces-1)*(i+1), holeRadius=holeRadius, angleBottom=-45, angleTop=-45, center=DIP+(end-DIP)/(numFaces-1)*(i+1))
        totalVertices.extend(transitionalVertices)
        
    for num in range(numFaces):
        offset = len(proximalStartVertices)*num
        for idx in range(resolution-3):
            faces.append((3, idx + offset, idx+1 + offset, idx+len(proximalStartVertices) + offset))
            faces.append((3, idx+len(proximalStartVertices) + offset, idx + 1 + len(proximalStartVertices) + offset, idx +1 + offset))
        offset = resolution -1 + len(proximalStartVertices)*num
        for idx in range(resolution-1):
            faces.append((3, idx + offset, idx+1 + offset, idx+len(proximalStartVertices) + offset))
            faces.append((3, idx+len(proximalStartVertices) + offset, idx + 1 + len(proximalStartVertices) + offset, idx +1 + offset))
        offset = resolution*2-1 + len(proximalStartVertices)*num
        for idx in range(resolution):
            faces.append((3, idx+ offset, idx + offset+1, idx +len(proximalStartVertices)+offset))
            faces.append((3, idx+len(proximalStartVertices) + offset, idx + 1 + len(proximalStartVertices) + offset, idx +1 + offset))

    for offset in range(numFaces):
        faces.append((3, resolution-3+offset*len(proximalStartVertices), 2*resolution-2+offset*len(proximalStartVertices), resolution-3+len(proximalStartVertices)+offset*len(proximalStartVertices)))
        faces.append((3, resolution-3+len(proximalStartVertices)+offset*len(proximalStartVertices), 2*resolution-2+len(proximalStartVertices)+offset*len(proximalStartVertices),  2*resolution-2+offset*len(proximalStartVertices)))
    for offset in range(numFaces):
        faces.append((3, offset*len(proximalStartVertices), resolution-1+offset*len(proximalStartVertices), len(proximalStartVertices)+offset*len(proximalStartVertices)))
        faces.append((3, resolution-1+offset*len(proximalStartVertices)+len(proximalStartVertices), resolution-1+offset*len(proximalStartVertices), len(proximalStartVertices)+offset*len(proximalStartVertices)))
    
    totalVertices.extend(distalEndVertices)
    for face in distalEndFaces:
        faces.append((3, face[1]+len(proximalStartVertices)*numFaces, face[2]+len(proximalStartVertices)*numFaces, face[3]+len(proximalStartVertices)*numFaces))

   
   
    # for idx in range(resolution-3):
    #     faces.append([3, idx, idx+1, idx+len(frontVertices)])
    #     faces.append([3, idx+len(frontVertices), idx+len(frontVertices)+1, idx+1])
    # offset = resolution-1
    # for idx in range(resolution-1):
    #     faces.append([3, idx+offset, idx+offset+1, idx+offset+len(frontVertices)])
    #     faces.append([3, idx+offset+len(frontVertices), idx+offset+len(frontVertices)+1, idx+offset+1])

    # faces.append([3, 0, len(frontVertices), len(frontVertices)+resolution-1])
    # faces.append([3, 0, len(frontVertices)+resolution-1, resolution-1])
    # faces.append([3, resolution-3, len(frontVertices)+resolution-3, len(frontVertices)+resolution+resolution-2])
    # faces.append([3, resolution-3, len(frontVertices)+resolution+resolution-2, resolution+resolution-2])

    # offset = resolution+resolution-1
    # for i in range(resolution+1):
    #     faces.append([3, offset+i, offset+((i+1)%(resolution+1)), len(frontVertices)+offset+(i%(resolution+1))])
    #     faces.append([3, offset+((i+1)%(resolution+1)), len(frontVertices)+offset+(i%(resolution+1)), len(frontVertices)+offset+((i+1)%(resolution+1))])
    mesh = pv.PolyData(totalVertices, faces).triangulate()
    return mesh

def move_and_align_mesh(mesh, new_center, new_align_vector, new_normal):
    otherNormal = np.cross(new_align_vector, new_normal)
    transformation_matrix = np.eye(4)
    A = [new_align_vector/np.linalg.norm(new_align_vector), new_normal/np.linalg.norm(new_normal), otherNormal/np.linalg.norm(otherNormal)]
    transformation_matrix[0:3, 0:3] =np.linalg.inv(A)
    mesh = mesh.transform(transformation_matrix)
    translation_vector = new_center - np.array([0,0,0], dtype=float)
    mesh = mesh.translate(translation_vector)
    return mesh

thingy = generateFingerPart(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius, PIP=np.array((0,0,0)), DIP=np.array((0,0,10)), end=np.array((0,5,15)))
# thingy = move_and_align_mesh(thingy, np.array([0,0,0], dtype=float), np.array([1,1,0], dtype=float), np.array([1,0,3], dtype=float))
# oldThingy = generateFingerPart(resolution=resolution, topRadii=topRadii, bottomRadii=bottomRadii, fingerBodyHeight=fingerBodyHeight, fingerBodyWidth=fingerBodyWidth, holeRadius=holeRadius, fingerBodyLength=fingerBodyLength)

pl = pv.Plotter()
pl.add_mesh(thingy)
pl.show()

6.0
4.666666666666667
3.3333333333333335


Widget(value="<iframe src='http://localhost:53052/index.html?ui=P_0x217b67501c0_1&reconnect=auto' style='width…

In [3]:
import numpy as np
import pyvista as pv

resolutionEnd=10
resolutionBody = 10
thickness = 2
endRadius = 3
length = 10
width = 15

def genConnector(endRadius, length, thickness, resolutionBody, resolutionEnd, width):
    def createConnector(endRadius, length, thickness, resolutionBody, resolutionEnd):
        vertices = []
        faces = []

        for i in range(resolutionBody+1):
            vertices.append((0, thickness/2, length/resolutionBody*i-length/2))
        for i in range(resolutionBody+1):
            vertices.append((0, -thickness/2, length/resolutionBody*i-length/2))
        for i in range(resolutionBody):
            faces.append((3, i, i + 1, i + 1 + resolutionBody + 1))
            faces.append((3, i + resolutionBody + 2, i + resolutionBody + 1, i))
        
        currTheta = np.pi-np.arcsin(thickness/2/endRadius)
        thetaInterval = 2*currTheta/(resolutionEnd+1)
        currTheta-=thetaInterval
        for _ in range(resolutionEnd):
            vertices.append((0, endRadius*np.sin(currTheta), endRadius*np.cos(currTheta)+length/2+endRadius))
            currTheta -= thetaInterval

        currTheta = np.arcsin(thickness/2/endRadius)
        thetaInterval = 2*(np.pi-currTheta)/(resolutionEnd+1)
        currTheta-=thetaInterval*2
        for _ in range(resolutionEnd):
            vertices.append((0, endRadius*np.sin(currTheta), endRadius*np.cos(currTheta)-length/2-endRadius))
            currTheta -= thetaInterval

        endCenter1 = (0, 0, (length/2+endRadius))
        vertices.append(endCenter1)
        endCenter2 = (0, 0, (-length/2-endRadius))
        vertices.append(endCenter2)
        for i in range(resolutionEnd-1):
            faces.append((3, i+2*resolutionBody+2, len(vertices)-2, i+2*resolutionBody+3))
            faces.append((3, i+2*resolutionBody+2+resolutionEnd, len(vertices)-1, i+2*resolutionBody+3+resolutionEnd))

        faces.append((3, len(vertices)-1, 0, resolutionBody+1))
        faces.append((3, len(vertices)-1, 0, len(vertices)-3))
        faces.append((3, len(vertices)-1, resolutionBody+1, len(vertices)-2-resolutionEnd))

        faces.append((3, len(vertices)-2, resolutionBody, 2*resolutionBody+1))
        faces.append((3, len(vertices)-2, 2*resolutionBody+1, 2*resolutionBody+1+resolutionEnd))
        faces.append((3, len(vertices)-2, resolutionBody, 2*resolutionBody+2))
        return vertices, faces
    totalVertices = []
    totalFaces = []
    verts1, faces1 = createConnector(endRadius=endRadius, length=length, thickness=thickness, resolutionBody=resolutionBody, resolutionEnd=resolutionEnd) 
    for vert in verts1:
        totalVertices.append((width/2, vert[1], vert[2]))
    totalFaces.extend(faces1)
    verts2, faces2 = createConnector(endRadius=endRadius, length=length, thickness=thickness, resolutionBody=resolutionBody, resolutionEnd=resolutionEnd) 
    for vert in verts2:
        totalVertices.append((-width/2, vert[1], vert[2]))
    for face in faces2:
        totalFaces.append((3, face[1]+len(verts1), face[2]+len(verts1), face[3]+len(verts1)))
    for vert in range(resolutionBody):
        totalFaces.append((3, vert, (vert+1), len(verts1)+(vert+1)))
        totalFaces.append((3, len(verts1)+(vert+1), len(verts1)+vert, vert))
        totalFaces.append((3, vert+resolutionBody+1, (vert+1)+resolutionBody+1, len(verts1)+(vert+1)+resolutionBody+1))
        totalFaces.append((3, len(verts1)+(vert+1)+resolutionBody+1, len(verts1)+vert+resolutionBody+1, vert+resolutionBody+1))
    for i in range(resolutionEnd-1):
        totalFaces.append((3, i+2*resolutionBody+2, i+2*resolutionBody+3, i+2*resolutionBody+2+len(verts1)))
        totalFaces.append((3, i+2*resolutionBody+3+len(verts1), i+2*resolutionBody+2+len(verts1), i+2*resolutionBody+3))
        
        totalFaces.append((3, i+2*resolutionBody+2+resolutionEnd, i+2*resolutionBody+3+resolutionEnd, i+2*resolutionBody+2+len(verts1)+resolutionEnd))
        totalFaces.append((3, i+2*resolutionBody+3+len(verts1)+resolutionEnd, i+2*resolutionBody+2+len(verts1)+resolutionEnd, i+2*resolutionBody+3+resolutionEnd))

    totalFaces.append((3, 2*resolutionBody+2, len(verts1)+resolutionBody, 2*resolutionBody+2+len(verts1)))
    totalFaces.append((3, 2*resolutionBody+2, resolutionBody, len(verts1)+resolutionBody))

    totalFaces.append((3, 2*resolutionBody+2+resolutionEnd-1, len(verts1)+2*resolutionBody+1,  2*resolutionBody+2+resolutionEnd-1+len(verts1)))
    totalFaces.append((3, 2*resolutionBody+1, len(verts1)+2*resolutionBody+1,  2*resolutionBody+2+resolutionEnd-1))

    totalFaces.append((3, len(verts1),0,  2*resolutionBody+resolutionEnd+resolutionEnd+1))
    totalFaces.append((3, 2*resolutionBody+resolutionEnd+resolutionEnd+1+len(verts1),len(verts1),  2*resolutionBody+resolutionEnd+resolutionEnd+1))

    totalFaces.append((3, resolutionBody+1, len(verts1)+resolutionBody+1,  2*resolutionBody+2+resolutionEnd))
    totalFaces.append((3, len(verts1)+2*resolutionBody+2+resolutionEnd, len(verts1)+resolutionBody+1,  2*resolutionBody+2+resolutionEnd))
    return pv.PolyData(totalVertices, totalFaces)

mesh = genConnector(endRadius=2, length=length, thickness=thickness, resolutionBody=resolutionBody, resolutionEnd=resolutionEnd, width=width)

pl = pv.Plotter()
# actor = pl.add_points(
#     np.array(verts), render_points_as_spheres=True, point_size=10.0
# )
pl.add_mesh(mesh)
pl.show()

Widget(value="<iframe src='http://localhost:53052/index.html?ui=P_0x217b1218b20_2&reconnect=auto' style='width…

In [64]:
df = df.apply(lambda x: x*2)
df

,Index,Middle,Third,Fourth
Names,,,,
METATOBASE,265.18,265.18,265.18,265.18
PIPTOMETA,60.38,66.44,57.44,45.78
DIPPIP,58.44,65.94,58.50,42.60
ENDDIP,63.90,67.62,67.68,61.40


In [1]:
import numpy as np
import pyvista as pv
from scipy.spatial import cKDTree

def genJointDisc(jointCenter, jointNormal, fingerDiskRadius=3, resolution=100):
    jointNormal = jointNormal / np.linalg.norm(jointNormal)
    jointBiNormal = np.cross(jointNormal, np.array([1, 0, 0]))
    jointBiNormal /= np.linalg.norm(jointBiNormal)
    circlePoints = []
    for theta in np.linspace(0, 2 * np.pi, resolution, endpoint=False):
        point = jointCenter + fingerDiskRadius * (jointNormal * np.cos(theta) + jointBiNormal * np.sin(theta))
        circlePoints.append(point)
    return np.array(circlePoints)

points = genJointDisc(np.array([1, 0, 0]), np.array([3, 6, 8]), fingerDiskRadius=5)
points = np.vstack((points, genJointDisc(np.array([2, 0, 0]), np.array([3, 6, 8]))))
tree = cKDTree(points)
print(points)
n = 50
grid = pv.UniformGrid(
    dimensions=(n, n, n),
    spacing=(10.0 / n, 10.0 / n, 10.0 / n),
    origin=(-5, -5, -5),
)
x, y, z = grid.points.T
distances, _ = tree.query(np.c_[x, y, z])
values = distances**2

iso_values = [2.0, 3.0, 4.0, 5.0]

for iso_value in iso_values:
    mesh = grid.contour([iso_value], scalars=values, method='marching_cubes')
    if mesh.n_points > 0:
        print(f"Found a suitable iso_value: {iso_value}")
        mesh.plot(smooth_shading=True, cmap="plasma", show_scalar_bar=False)
        break
else:
    print("No suitable iso_value found. Try adjusting the range or resolution.")


[[ 2.43673943  2.87347886  3.83130514]
 [ 2.43390435  3.11897078  3.63537338]
 [ 2.42541031  3.35215355  3.42509445]
 [ 2.41129082  3.5721069   3.20129825]
 [ 2.39160162  3.77796278  2.96486798]
 [ 2.3664204   3.96890877  2.71673674]
 [ 2.33584654  4.14419128  2.45788377]
 [ 2.3000007   4.30311857  2.18933066]
 [ 2.25902436  4.44506341  1.91213727]
 [ 2.21307922  4.56946562  1.62739754]
 [ 2.16234661  4.67583424  1.33623521]
 [ 2.10702676  4.76374947  1.03979938]
 [ 2.04733797  4.83286436  0.73925993]
 [ 1.98351582  4.88290614  0.43580296]
 [ 1.91581218  4.91367733  0.13062608]
 [ 1.84449425  4.92505647 -0.17506632]
 [ 1.76984348  4.91699867 -0.48006782]
 [ 1.6921545   4.88953572 -0.78317471]
 [ 1.6117339   4.842776   -1.08319077]
 [ 1.52889906  4.77690406 -1.37893197]
 [ 1.4439769   4.69217986 -1.66923115]
 [ 1.35730257  4.58893778 -1.95294264]
 [ 1.26921812  4.46758525 -2.22894676]
 [ 1.1800712   4.3286012  -2.49615424]
 [ 1.09021362  4.17253414 -2.75351055]
 [ 1.          4.        

c:\Users\Katie\anaconda3\envs\prosthetics3.10\lib\site-packages\pyvista\core\grid.py:912: PyVistaDeprecationWarning: `UniformGrid` is deprecated. Use `ImageData` instead.
  warnings.warn(


Widget(value="<iframe src='http://localhost:51909/index.html?ui=P_0x109650190c0_0&reconnect=auto' style='width…

In [72]:
import numpy as np
import pyvista as pv
points = np.random.random((10, 3))
pl = pv.Plotter()
actor = pl.add_points(
    points, render_points_as_spheres=False, point_size=100.0
)
pl.show()

Widget(value="<iframe src='http://localhost:64776/index.html?ui=P_0x24cecb53760_49&reconnect=auto' style='widt…